In [18]:
import sort_RAFT_table as srt
import pandas as pd
import numpy as np
import plotly.express as px
import re
from scipy.optimize import curve_fit

In [19]:
# ToDo: Kinetik Kurven erstellen,
#  90% oder max als max gemessen als conversion, als Beschreibung für diese nehmen,
#  in ist Monomer und Mn dann
#       sortieren nach Umsatz dahingehend dann nach kürzester Zeit
#       Tabellenausgabe dazu Kinetik Kurve, reaktionsbedingungen (LM, Raft agent, Zeit...)
#  am besten als interaktive website Tabelle.

# get all conversion values,
conversion_list = []
for column in srt.df.columns:
    if "conversion" in column:
        conversion_list.append(column)
print(conversion_list)

# get the hours from the headers with regex
hours_list = []
two_digit_regex = r"\d+"
for column in conversion_list:
    hours_list.append(int(re.findall(two_digit_regex, column)[0]))

print(hours_list)

['t0h-conversion', 't1h-conversion', 't2h-conversion', 't4h-conversion', 't8h-conversion', 't15h-conversion']
[0, 1, 2, 4, 8, 15]


In [20]:
def change_timeformat(time_format):
    h_m_s = str(time_format).split(":")
    m_format = int(h_m_s[0]) * 60 + int(h_m_s[1]) + int(h_m_s[2]) / 60
    return m_format

def change_timeformat_h(time_format):
    h_m_s = str(time_format).split(":")
    h_format = int(h_m_s[0]) + (int(h_m_s[1]) + int(h_m_s[2]) / 60) /60
    return h_format

# creating a table as a lookup to correct all sample measurement times
exact_times = pd.read_excel(srt.INPUT_FILE_PATH, sheet_name="exact sampling times")

time_correction_df = pd.DataFrame(data=exact_times.iloc[3:12, 3:])
time_correction_df.columns = exact_times.loc[2][3:]

time_correction_df.reset_index(inplace=True, drop=True)
ext_time_corr_df_data =[]
for row in time_correction_df.iterrows():
    reactor_nr = row[1]["Reactor"]
    if type(reactor_nr) == int or reactor_nr == "closing of reactors":
        row[1]["Reactor"] = str(reactor_nr)
        ext_time_corr_df_data.append(row[1])
    else:
        reactor_nr = (row[1]["Reactor"]).split("+")
        for nr in reactor_nr:
            row[1]["Reactor"] = nr
            ext_time_corr_df_data.append(row[1].copy())
ext_time_corr_df = pd.DataFrame(data=ext_time_corr_df_data, columns=exact_times.loc[2][3:])
ext_time_corr_df.reset_index(drop=True, inplace=True)
print("The timer was reset to 0 after the first closing of reactors in the \"t = 0\"-sampling-step")
ext_time_corr_df.columns = ["Reactor", 0, 1, 2, 4, 6, 8, 10, 15]

# change time format to minutes and set
time_cols = ext_time_corr_df.columns.difference(["Reactor"])
ext_time_corr_df[time_cols] = ext_time_corr_df[time_cols].apply(lambda x: [change_timeformat_h(d) for d in x])
ext_time_corr_df[0] = ext_time_corr_df[0].apply(lambda x: 0 )

# set index to reactor
ext_time_corr_df.set_index("Reactor", inplace=True)

# Sampling 6h and 10h were only used for sec not NMR -> no conversion for these.
skip_hours_mask = [True if h not in hours_list else False for h in ext_time_corr_df.columns]
ext_time_corr_df.drop(columns=time_cols[skip_hours_mask], inplace=True)

ext_time_corr_df



The timer was reset to 0 after the first closing of reactors in the "t = 0"-sampling-step


,0,1,2,4,8,15
Reactor,,,,,,
1,0,1.005556,2.011111,4.018056,8.016667,15.016667
2,0,1.005556,2.011111,4.018056,8.016667,15.016667
3,0,1.073611,2.068056,4.077778,8.090278,15.098611
4,0,1.073611,2.068056,4.077778,8.090278,15.098611
5,0,1.131944,2.125000,4.143056,8.180556,15.200000
6,0,1.131944,2.125000,4.143056,8.180556,15.200000
7,0,1.191667,2.184722,4.205556,8.256944,15.291667
8,0,1.191667,2.184722,4.205556,8.256944,15.291667
9,0,1.251389,2.241667,4.273611,8.338889,15.381944


In [21]:
# srt.df.columns
ext_time_corr_df.loc["15"]


0      0.000000
1      1.437500
2      2.430556
4      4.475000
8      8.581944
15    15.658333
Name: 15, dtype: float64

In [22]:
# plotting the kinetic curves of the RAFT polymerization
# generate a reformatted table with a entries per sample analysis taken described with a column of the right time

# create a dataframe with just the information needed to append to
kinetic_curves = []
for index, polymerisation_kinetic in srt.df.iterrows():
    # get the right hours list
    # polymerisation_kinetic["reactor"]

    kinetic_curve_entries = pd.DataFrame(
        {"conversion" : polymerisation_kinetic[conversion_list], "time" : hours_list, "reactor" : polymerisation_kinetic["reactor"]})
    kinetic_curve_entries["experiment"] = str(index)+"_"+polymerisation_kinetic["monomer"]+"_"+polymerisation_kinetic["RAFT-Agent"]+"_"+polymerisation_kinetic["solvent"]

    # the times are dependent on the current reactor, get current
    current_reactor_nr = str(polymerisation_kinetic["reactor"])
    current_time_list = ext_time_corr_df.loc[current_reactor_nr]
    kinetic_curve_entries["time"] = list(current_time_list)

    kinetic_curves.append(kinetic_curve_entries)

kinetic_curves_df = pd.concat(kinetic_curves)

In [53]:
len(kinetic_curves) # was 325 before getting away the average below 0.1
# kinetic_curves[1]["experiment"].iloc[1]
# px.line(kinetic_curves_df[:20], x="time", y="conversion", color="experiment")

325

In [24]:
# and fit curve and the derivation
# find the maximum of the derivation

'''
deprecated: more theoretical accurate version but practically worse applicable
    # L is responsible for scaling the output range from [0,1] to [0,L]
    # b
    # k is responsible for scaling the input, which remains in (-inf,inf)
    # x0
def sigmoid (x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0)))+b
    return (y)


def sigmoid_derivative(x, L ,x0, k, b): # ToDo: check if this is the correct derivative
    y = (L*k*np.exp(-k*(x-x0)))/((np.exp(-k*(x-x0))+1)**2)
    return (y)
'''
def neg_growth(x, L, k):
    y = L * (1 - np.exp(k*(-x)))
    return y

def neg_growth_derivative(x, L, k):
    y = L * k * np.exp(k*(-x))
    return y

def linear_growth(x, m):
    y = m * x
    return y

def linear_growth_derivative(x, m):
    return m

xrange = np.arange(-0.2, 5, 0.1)
example_fig = px.line()
for testparams in ([1,1], [2,1], [1,2]):
    example_fig.add_scatter(x=xrange, y=neg_growth(xrange, *testparams))
    example_fig.add_scatter(x=xrange, y=neg_growth_derivative(xrange, *testparams), opacity=0.5, line=dict(dash="dot"))

print("visualisation of the functions")
# example_fig.show()

visualisation of the functions


In [65]:
fig = px.line(title="Kinetic Curve Fit", labels={"x":"time", "y":"conversion"})
hard_to_fit = [] # array for kinetic fits that meet a filter criteria further down the code
colors = px.colors.qualitative.Plotly # set up a simple color palette
extended_xdata = np.linspace(-1, 16.5, 100) # x data array for

# create new dataframe with kinetics per row
kinetic_curve_columns = ["Experiment", "max_conversion", "reaction_end", "fit_params", "params error", "squared_error"]
kinetics_df = pd.DataFrame(columns=kinetic_curve_columns)

def add_fits_to_plot(fit_func, fit_func_derivative, fit_func_params, marker_dic):
    fig.add_scatter(
        x=extended_xdata, y=fit_func(extended_xdata, *fit_func_params),
        opacity=1, line=dict(dash="dot"), name=f"{fit_func.__name__} fit", marker=marker_dic)
    print(fit_func_params)
    fig.add_scatter(
        x=extended_xdata, y=fit_func_derivative(extended_xdata, *fit_func_params),
        opacity=0.3, line=dict(dash="dash"), name=f"{fit_func_derivative.__name__}", marker=marker_dic)

for idx, kinetic_curve in enumerate(kinetic_curves):
    ydata = kinetic_curve["conversion"].values
    xdata = kinetic_curve["time"].values

    # choosing a few test function
    # if not idx in (1, 2):
    #     continue

    # fitting section
    p0 = [max(ydata), 0.1] # this is a mandatory initial guess
    cv_data = curve_fit(f=neg_growth, xdata=xdata, ydata=ydata, p0=p0, nan_policy="omit", maxfev=800*10)
    popt, pcov = cv_data
        # fit with linear growth
    p0_l = [max(ydata)/7]
    cv_data_l = curve_fit(f=linear_growth, xdata=xdata, ydata=ydata, p0=p0_l, nan_policy="omit")
    popt_l, pcov_l = cv_data_l

    # calculate the squared error of fit and data points
    y_st =[]
    x_st = []
    for y_val, x_val in zip(ydata, xdata):
        if np.isnan(y_val) or np.isnan(x_val):
            continue
        y_st.append(y_val)
        x_st.append(x_val)
    y_st = np.array(y_st)
    x_st = np.array(x_st)
    squared_error = np.sum((y_st - neg_growth(x_st, *popt))**2)
    # print("{:.6f}".format(squared_error), str(idx) + ": The optimal parameters are " + " and ".join("%.3f" % x for x in popt) + ". The variance is " + " and respectively ".join("{:.3f} {:.3f}".format(x, y) for x, y in pcov) + ".")

    # ToDo: fill new dataframe
    new_row = pd.DataFrame({"Experiment":kinetic_curve["experiment"].iloc[1], "max_conversion":max(ydata),
                        "reaction_end":"yet to calc", "fit_params":[popt], "params error":[pcov], "squared_error":squared_error})
    kinetics_df = pd.concat([kinetics_df, new_row])

    # filter for inspection
    # if not ((popt[1]) < 0.001).any(): continue
    # hard_to_fit.append(idx)

''' comment out if no plot is needed.
    marker_dict = dict(color=colors[idx%len(colors)]) # set same colors per kinetic
    fig.add_scatter(x=xdata, y=ydata, mode="lines+markers", opacity=1, name=kinetic_curve["experiment"].iloc[1], marker=marker_dict)
    add_fits_to_plot(neg_growth, neg_growth_derivative, popt, marker_dict)
    # add_fits_to_plot(linear_growth, linear_growth_derivative, [popt_l], marker_dict)


fig.update_layout(
    yaxis=dict(
        range=[-0.1,1]
    ),
    xaxis_title="Time [h]",
    yaxis_title="Conversion [%]"
)
fig.show()
# '''
print(hard_to_fit, len(hard_to_fit)/len(kinetic_curves))



C:\Users\Nex\AppData\Local\Temp\ipykernel_4548\480568675.py:52: FutureWarning:

The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.



[] 0.0


In [66]:
kinetics_df

,Experiment,max_conversion,reaction_end,fit_params,params error,squared_error
0,0_6_A_DMF,0.516574,yet to calc,"[0.7358519036455098, 0.07920353070346808]","[[0.008884156829792648, -0.0014823097511405452...",1.192962e-03
0,1_6_A_DMSO,0.697321,yet to calc,"[0.8024976784785777, 0.13316078717682855]","[[0.0007840187926938868, -0.000239964466669024...",5.936409e-04
0,2_6_A_Tol,0.323281,yet to calc,"[1.06738558144129, 0.02388860357275495]","[[0.1238845064146147, -0.003219055326314272], ...",1.747652e-04
0,3_7_A_DMF,0.565125,yet to calc,"[0.9530881797253814, 0.059934107597394565]","[[0.009145537883212228, -0.0008158734783419188...",4.525607e-04
0,4_7_A_DMSO,0.790565,yet to calc,"[1.0926739688051226, 0.08391032775996919]","[[0.0032852289698699225, -0.000420949463106630...",3.205737e-04
...,...,...,...,...,...,...
0,320_9_E_DMF,0.734328,yet to calc,"[0.7964859207947647, 0.16527052933672504]","[[8.752620439103308e-06, -3.533527798428389e-0...",1.305254e-05
0,321_9_E_DMSO,0.546547,yet to calc,"[0.9296798050540719, 0.20437130007240129]","[[2.472470977510398e-05, -7.746357491004986e-0...",4.779775e-07
0,322_9_E_Tol,0.360000,yet to calc,"[0.3832365304238183, 0.19767543775558907]","[[0.0005599788887519543, -0.000578943653394831...",1.283833e-03
0,323_10_E_DMF,0.840426,yet to calc,"[0.928939140208845, 0.15452012713452215]","[[0.001383263577535527, -0.0004407239685077047...",1.803572e-03
